# 볼린저 밴드 함수화
1. 첫 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 결측치,이상치 제거한다.
    2. 컬럼을 종가만 두고 모두 삭제
    3. 이동평균선, 상단밴드, 하단밴드 생성
    
2. 두 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 거래 내역이라는 파생 변수 생성 값은  ""
    2. 볼린저 밴드의 거래 내역을 추가
    3. 거래 내역을 출력(print)

3. 세 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익율 파생변수 생성 값은 "1"
    2. 판매를 한 날의 수익율 변경
    3. 누적 수익율을 계산하여 새로운 파생변수를 생성
    4. 최종 누적 수익율을 출력(print)

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
# 함수 1
def bollinger1(df,col):
    df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1)]
    price_df=df[[col]]  # price_df라는 변수 1-2
    price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
    price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
    price_df['lb']=price_df['center']-(2*price_df[col].rolling(20).std())   # 하단 밴드 파생변수
    

    return price_df





In [5]:
df = pd.read_csv('../csv/GM.csv', index_col='Date')
df=bollinger1(df,"Close")
df

C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,Close,center,ub,lb
Date,,,,
2010-11-18,34.189999,NaN,NaN,NaN
2010-11-19,34.259998,NaN,NaN,NaN
2010-11-22,34.080002,NaN,NaN,NaN
2010-11-23,33.250000,NaN,NaN,NaN
2010-11-24,33.480000,NaN,NaN,NaN
...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451
2019-06-19,36.779999,35.43650,37.173130,33.699870
2019-06-20,36.959999,35.50700,37.372713,33.641286


In [6]:
# 함수 2
def bollinger2(price_df,col):
    price_df['trade']=""    # 거래 내역이라는 파생변수

    for i in price_df.index:
    # 상단 밴드보다 종가가 높은 경우
        if price_df.loc[i,col]> price_df.loc[i,"ub"]:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #매도
                price_df.loc[i,'trade']=''
            else:
                price_df.loc[i,'trade']=''
            
            # 하단 밴드보다 종가가 낮은 경우
        elif price_df.loc[i,col]< price_df.loc[i,'lb']:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=="buy":
                # 구매 상태를 유지
                price_df.loc[i,'trade']= 'buy'
            else:
                price_df.loc[i,'trade']='buy'
        else:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #구매 상태를 유지
                price_df.loc[i,'trade']='buy'
            else:
                price_df.loc[i,'trade']=''

    return price_df



In [7]:
df=bollinger2(df,"Close")
df

,Close,center,ub,lb,trade
Date,,,,,
2010-11-18,34.189999,NaN,NaN,NaN,
2010-11-19,34.259998,NaN,NaN,NaN,
2010-11-22,34.080002,NaN,NaN,NaN,
2010-11-23,33.250000,NaN,NaN,NaN,
2010-11-24,33.480000,NaN,NaN,NaN,
...,...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451,buy
2019-06-19,36.779999,35.43650,37.173130,33.699870,buy
2019-06-20,36.959999,35.50700,37.372713,33.641286,buy


In [8]:
# 함수 3
def bollinger3(N_df,col):# 파생변수 N_df
    
    rtn=1.0
    N_df["return"]=1       # "return"이라는 파생변수 추가
    buy=0.0
    sell=0.0
    for i in N_df.index:
        # 구매가를 출력
        if (N_df.shift(1).loc[i,'trade']=="") and (N_df.loc[i,'trade']=="buy"):
            buy =N_df.loc[i,col]
            # print('진입일 :',i, '구매가격 :',buy)
        # 판매가를 출력
        elif (N_df.shift(1).loc[i,'trade']=="buy") and (N_df.loc[i,'trade']==""):
            sell=N_df.loc[i,col]
            rtn =(sell-buy)/buy +1
            N_df.loc[i,'return'] =rtn
            # print('판매일 :',i, "판매가격 :",sell, '수익율 :', rtn)

        # 구매가, 판매가 초기화
        if N_df.loc[i,'trade']=='':
            buy=0.0
            sell=0.0

    # 누적 수익율

    acc_rtn =1.0     # 누적 수익률

    for i in N_df.index:
        rtn=N_df.loc[i,'return']
        acc_rtn *=rtn
        N_df.loc[i,"acc_rtn"]=acc_rtn


    print('누적 수익율:', acc_rtn)
    
    return N_df

In [9]:
bollinger3(df,"Close")

누적 수익율: 1.1480914103747033


,Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-11-18,34.189999,NaN,NaN,NaN,,1.0,1.000000
2010-11-19,34.259998,NaN,NaN,NaN,,1.0,1.000000
2010-11-22,34.080002,NaN,NaN,NaN,,1.0,1.000000
2010-11-23,33.250000,NaN,NaN,NaN,,1.0,1.000000
2010-11-24,33.480000,NaN,NaN,NaN,,1.0,1.000000
...,...,...,...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451,buy,1.0,1.148091
2019-06-19,36.779999,35.43650,37.173130,33.699870,buy,1.0,1.148091
2019-06-20,36.959999,35.50700,37.372713,33.641286,buy,1.0,1.148091


In [10]:
df = pd.read_csv('../csv/GDX.csv', index_col='Date')
df=bollinger1(df,'Close')
df=bollinger2(df,"Close")
bollinger3(df,"Close")



C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

누적 수익율: 2.687593657120985


,Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2006-05-22,37.230000,NaN,NaN,NaN,,1.0,1.000000
2006-05-23,37.959999,NaN,NaN,NaN,,1.0,1.000000
2006-05-24,36.520000,NaN,NaN,NaN,,1.0,1.000000
2006-05-25,38.320000,NaN,NaN,NaN,,1.0,1.000000
2006-05-26,38.549999,NaN,NaN,NaN,,1.0,1.000000
...,...,...,...,...,...,...,...
2019-06-18,23.670000,22.023000,24.439514,19.606486,,1.0,2.687594
2019-06-19,24.000000,22.187500,24.674568,19.700432,,1.0,2.687594
2019-06-20,25.049999,22.422000,25.063192,19.780808,,1.0,2.687594


In [11]:
df = pd.read_csv('../csv/GM.csv', index_col='Date')
df=bollinger1(df,"Close")
df

C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\661784885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,Close,center,ub,lb
Date,,,,
2010-11-18,34.189999,NaN,NaN,NaN
2010-11-19,34.259998,NaN,NaN,NaN
2010-11-22,34.080002,NaN,NaN,NaN
2010-11-23,33.250000,NaN,NaN,NaN
2010-11-24,33.480000,NaN,NaN,NaN
...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451
2019-06-19,36.779999,35.43650,37.173130,33.699870
2019-06-20,36.959999,35.50700,37.372713,33.641286


In [12]:
#### bollinger1에 매개변수 2개, bollinger2에 매개변수 1개, bollinger3에 매개변수 1개 ####
#### class를 써서 처음 받는 "Adj_close"를 class변수에 넣어준다면 쉬움 ####

# 함수 1
def bollinger1(df,col):
    df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1)]
    # df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1),[]]
    price_df=df[[col]]  # price_df라는 변수 1-2
    price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
    price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
    price_df['lb']=price_df['center']-(2*price_df[col].rolling(20).std())   # 하단 밴드 파생변수



    return price_df


# 함수 2
def bollinger2(price_df):
    price_df['trade']=""    # 거래 내역이라는 파생변수
    price_df2=price_df.iloc[:,0]

    for i in price_df.index:
    # 상단 밴드보다 종가가 높은 경우
        if price_df2[i]> price_df.loc[i,"ub"]:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #매도
                price_df.loc[i,'trade']=''
            else:
                price_df.loc[i,'trade']=''
            
            # 하단 밴드보다 종가가 낮은 경우
        elif price_df2[i]< price_df.loc[i,'lb']:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=="buy":
                # 구매 상태를 유지
                price_df.loc[i,'trade']= 'buy'
            else:
                price_df.loc[i,'trade']='buy'
        else:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #구매 상태를 유지
                price_df.loc[i,'trade']='buy'
            else:
                price_df.loc[i,'trade']=''

    return price_df



# 함수 3
def bollinger3(N_df):# 파생변수 N_df

    N_df2=N_df.iloc[:,0]
    rtn=1.0
    N_df["return"]=1       # "return"이라는 파생변수 추가
    buy=0.0
    sell=0.0
    for i in N_df.index:
        # 구매가를 출력
        if (N_df.shift(1).loc[i,'trade']=="") and (N_df.loc[i,'trade']=="buy"):
            buy =N_df2[i]
            # print('진입일 :',i, '구매가격 :',buy)
        # 판매가를 출력
        elif (N_df.shift(1).loc[i,'trade']=="buy") and (N_df.loc[i,'trade']==""):
            sell=N_df2[i]
            rtn =(sell-buy)/buy +1
            N_df.loc[i,'return'] =rtn
            # print('판매일 :',i, "판매가격 :",sell, '수익율 :', rtn)

        # 구매가, 판매가 초기화
        if N_df.loc[i,'trade']=='':
            buy=0.0
            sell=0.0

    # 누적 수익율

    acc_rtn =1.0     # 누적 수익률

    for i in N_df.index:
        rtn=N_df.loc[i,'return']
        acc_rtn *=rtn
        N_df.loc[i,"acc_rtn"]=acc_rtn


    print('누적 수익율:', acc_rtn)
    return N_df

In [13]:
df = pd.read_csv('../csv/GDX.csv', index_col='Date')
df=bollinger1(df,'Close')
df=bollinger2(df)
bollinger3(df)


C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\2989463668.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\2989463668.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\2989463668.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy o

누적 수익율: 2.687593657120985


,Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2006-05-22,37.230000,NaN,NaN,NaN,,1.0,1.000000
2006-05-23,37.959999,NaN,NaN,NaN,,1.0,1.000000
2006-05-24,36.520000,NaN,NaN,NaN,,1.0,1.000000
2006-05-25,38.320000,NaN,NaN,NaN,,1.0,1.000000
2006-05-26,38.549999,NaN,NaN,NaN,,1.0,1.000000
...,...,...,...,...,...,...,...
2019-06-18,23.670000,22.023000,24.439514,19.606486,,1.0,2.687594
2019-06-19,24.000000,22.187500,24.674568,19.700432,,1.0,2.687594
2019-06-20,25.049999,22.422000,25.063192,19.780808,,1.0,2.687594


In [14]:
#### bollinger1에 매개변수 2개, bollinger2에 매개변수 1개, bollinger3에 매개변수 1개 ####
#### class를 써서 처음 받는 "Adj_close"를 class변수에 넣어준다면 쉬움 ####
#### start, end의 매개변수를 추가해서 필요한 만큼 데이터를 받는다 ####

# 함수 1
def bollinger1(df,col,start,end):
    # 인덱스를 시계열로 변경한다
    df.index=pd.to_datetime(df.index)
    # start,end를 시계열로 변경한다
    start=datetime.strptime(start,'%Y%m%d').isoformat()
    end=datetime.strptime(end,'%Y%m%d').isoformat()
    # 데이터를 시작시간부터 종료시간까지 필터
    df=df.loc[start:end]
    df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1)]
    # df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1),[]]
    price_df=df[[col]]  # price_df라는 변수 1-2
    price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
    price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
    price_df['lb']=price_df['center']-(2*price_df[col].rolling(20).std())   # 하단 밴드 파생변수



    return price_df


# 함수 2
def bollinger2(price_df):
    #기준이 되는 컬럼이 무엇인가?
    # 기준이 되는 컬럼은 컬럼 중 첫번째  ########################## 교수님 풀이 #####################---> 기준을 columns[0]으로 잡아서 비교
    col=price_df.columns[0]
    price_df['trade']=""    # 거래 내역이라는 파생변수
    

    for i in price_df.index:
    # 상단 밴드보다 종가가 높은 경우
    
        if price_df.loc[i,col]> price_df.loc[i,"ub"]:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #매도
                price_df.loc[i,'trade']=''
            else:
                price_df.loc[i,'trade']=''
            
            # 하단 밴드보다 종가가 낮은 경우
        elif price_df.loc[i,col]< price_df.loc[i,'lb']:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=="buy":
                # 구매 상태를 유지
                price_df.loc[i,'trade']= 'buy'
            else:
                price_df.loc[i,'trade']='buy'
        else:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #구매 상태를 유지
                price_df.loc[i,'trade']='buy'
            else:
                price_df.loc[i,'trade']=''

    return price_df



# 함수 3
def bollinger3(N_df):# 파생변수 N_df

    col=N_df.columns[0]
    rtn=1.0
    N_df["return"]=1       # "return"이라는 파생변수 추가
    buy=0.0
    sell=0.0
    for i in N_df.index:
        # 구매가를 출력
        if (N_df.shift(1).loc[i,'trade']=="") and (N_df.loc[i,'trade']=="buy"):
            buy =N_df.loc[i,col]
            # print('진입일 :',i, '구매가격 :',buy)
        # 판매가를 출력
        elif (N_df.shift(1).loc[i,'trade']=="buy") and (N_df.loc[i,'trade']==""):
            sell=N_df.loc[i,col]
            rtn =(sell-buy)/buy +1
            N_df.loc[i,'return'] =rtn
            # print('판매일 :',i, "판매가격 :",sell, '수익율 :', rtn)

        # 구매가, 판매가 초기화
        if N_df.loc[i,'trade']=='':
            buy=0.0
            sell=0.0

    # 누적 수익율

    acc_rtn =1.0     # 누적 수익률

    for i in N_df.index:
        rtn=N_df.loc[i,'return']
        acc_rtn *=rtn
        N_df.loc[i,"acc_rtn"]=acc_rtn


    print('누적 수익율:', acc_rtn)
    
    return N_df

In [32]:
df = pd.read_csv('../csv/GM.csv', index_col='Date')
df=bollinger1(df,'Close',"20110101","20150101")
df=bollinger2(df)
bollinger3(df)

C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\2165885399.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\2165885399.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_25872\2165885399.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy 

누적 수익율: 0.86209094812149


,Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2011-01-03,37.060001,NaN,NaN,NaN,,1.0,1.000000
2011-01-04,37.900002,NaN,NaN,NaN,,1.0,1.000000
2011-01-05,38.070000,NaN,NaN,NaN,,1.0,1.000000
2011-01-06,38.900002,NaN,NaN,NaN,,1.0,1.000000
2011-01-07,38.980000,NaN,NaN,NaN,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,33.430000,32.5620,34.464492,30.659507,buy,1.0,0.862091
2014-12-26,33.730000,32.6450,34.601199,30.688801,buy,1.0,0.862091
2014-12-29,34.599998,32.7035,34.821803,30.585196,buy,1.0,0.862091


In [71]:
import bollinger as bg
import imp
imp.reload(bg)


<module 'bollinger' from 'c:\\Users\\dgh06\\OneDrive\\문서\\GitHub\\python-work\\230413\\bollinger.py'>

In [63]:
df =pd.read_csv("../csv/MSFT.csv",index_col='Date')


In [64]:
#1번 함수 실행
data1=bg.bollinger1(df,'Adj Close','20100101','20190101')
#data1

c:\Users\dgh06\OneDrive\문서\GitHub\python-work\230413\bollinger.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.index=pd.to_datetime(self.df.index)
c:\Users\dgh06\OneDrive\문서\GitHub\python-work\230413\bollinger.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # start,end를 시계열로 변경한다
c:\Users\dgh06\OneDrive\문서\GitHub\python-work\230413\bollinger.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [66]:
# 2번 함수 실행
data2=bg.bollinger2(data1)

In [67]:
# 3번 함수 실행
bg.bollinger3(data2)

누적 수익율: 2.619028014666416


,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,24.525019,24.056307,24.964478,23.148135,,1.0,1.000000
2010-01-05,24.532942,24.095134,25.015633,23.174635,,1.0,1.000000
2010-01-06,24.382378,24.133962,25.032815,23.235109,,1.0,1.000000
2010-01-07,24.128809,24.168828,25.004852,23.332803,,1.0,1.000000
2010-01-08,24.295214,24.206467,24.989876,23.423058,,1.0,1.000000
...,...,...,...,...,...,...,...
2018-12-24,93.384033,105.437251,114.470702,96.403801,buy,1.0,2.619028
2018-12-26,99.763077,105.144093,114.525564,95.762621,buy,1.0,2.619028
2018-12-27,100.378166,104.848455,114.447882,95.249027,buy,1.0,2.619028


In [78]:
#### 클래스 부르기

In [113]:
import bollinger as bg
import imp
imp.reload(bg)


<module 'bollinger' from 'c:\\Users\\dgh06\\OneDrive\\문서\\GitHub\\python-work\\230413\\bollinger.py'>

In [114]:
# 클래스 생성

data=pd.read_csv("../csv/AAPL.csv",index_col='Date')

In [115]:
# 클래스 생성
Class_a=bg.Invest(data,"Adj Close")

In [116]:
Class_a.col

'Adj Close'

In [117]:
Class_a.bollinger('20100101','20150101')

누적 수익율: 1.1027174718391783


,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,,1.0,1.000000
2010-01-05,26.829010,25.169503,27.288098,23.050908,,1.0,1.000000
2010-01-06,26.402260,25.307290,27.366449,23.248130,,1.0,1.000000
2010-01-07,26.353460,25.436879,27.410937,23.462821,,1.0,1.000000
2010-01-08,26.528664,25.525609,27.529742,23.521475,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy,1.0,1.102717
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy,1.0,1.102717
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy,1.0,1.102717


In [118]:
Class_a.df


,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,,1.0,1.000000
2010-01-05,26.829010,25.169503,27.288098,23.050908,,1.0,1.000000
2010-01-06,26.402260,25.307290,27.366449,23.248130,,1.0,1.000000
2010-01-07,26.353460,25.436879,27.410937,23.462821,,1.0,1.000000
2010-01-08,26.528664,25.525609,27.529742,23.521475,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy,1.0,1.102717
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy,1.0,1.102717
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy,1.0,1.102717
